In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Como determinaremos a quantidade de calorias ideais?

A equação de Harris-Benedict é uma fórmula que utiliza a altura, peso, idade e sexo de determinado indivíduo para calcular a sua taxa metabólica basal(TMB)

**1º Passo:**
Determinar a taxa metabólica basal (TMB)

Homens: TMB = 66 + (13.7 x peso em quilogramas) + (5 x altura em centímetros) – (6.8 x idade em anos)

Mulheres: TMB = 655 + (9.6 X peso em quilogramas) + (1.8 X altura em centímetros) – (4.7 x idade em anos).

**2º Passo:**
Multiplicar o valor da TMB pelo fator de atividade física adequado

- Sedentário – pouco ou nenhum exercício (TMB x 1.2)

- Pouco ativo – exercício/esporte leve 1-3 dias/semana (TMB x 1.375)

- Moderadamente ativo – exercício/esporte moderado 3-5 dias/semana (TMB x 1.55)

- Muito ativo – exercício/esporte pesado 6-7 dias/semana (TMB x 1.725)

- Extremamente ativo – exercício/esporte muito pesado e trabalho físico intenso diariamente ou treino de 2x ao dia (TMB x 1.9)

Este cálculo vai te dar um valor estimado das necessidades energéticas diárias do seu paciente. Contudo, não se esqueça que se trata apenas de uma estimativa e que deve ser usada como um guia e não como uma regra.

Deverá trabalhar com cada paciente para garantir que suas necessidades nutricionais e objetivos de saúde sejam atendidos, mas pode ser necessário aumentar ou diminuir esse valor dependendo de como eles se sentem.